<a href="https://colab.research.google.com/github/Sobakons/Mnist/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [ ]:
# Загружаем набор данных MNIST
train_dataset = torchvision.datasets.MNIST(root='./data',  train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=torchvision.transforms.ToTensor(), download=True)

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) #слой с 1 входным каналом, 32 выходными каналами
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2) # помогает уменьшить размерность данных, уменьшить вычислительные затраты и предотвратить переобучение
        self.dropout1 = nn.Dropout2d(0.25) #создает слой исключения с вероятностью 0.25. Исключение — это метод регуляризации, который случайным образом исключает некоторые нейроны во время обучения, что помогает снизить переобучение
        self.dropout2 = nn.Dropout2d(0.5) #аналогично только вероятность 0.5
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):  #принимает входной сигнал x и применяет ряд операций, определенных слоями в методе __init__.
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device(type='cpu')

In [ ]:
# Создаем экземпляр модели
model = Classifier()

# Переместим модель на графический процессор
model.to(device)
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
         MaxPool2d-2           [-1, 32, 14, 14]               0
         Dropout2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
         MaxPool2d-5             [-1, 64, 7, 7]               0
         Dropout2d-6             [-1, 64, 7, 7]               0
            Linear-7                  [-1, 128]         401,536
            Linear-8                   [-1, 10]           1,290
Total params: 421,642
Trainable params: 421,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.43
Params size (MB): 1.61
Estimated Total Size (MB): 2.04
----------------------------------------------------------------


In [ ]:
# Определим функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch_size = 100
num_epochs = 10
val_percent = 0.2 # процент данных, использованных для проверки
val_size = int(val_percent * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
losses = []
accuracies = []
val_losses = []
val_accuracies = []

In [ ]:
# Обучаем модель
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
    acc = (predicted == labels).sum().item() / labels.size(0)
    accuracies.append(acc)
    losses.append(loss.item())
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
        total = labels.size(0)
        correct = (predicted == labels).sum().item()
        val_acc += correct / total
        val_accuracies.append(acc)
        val_losses.append(loss.item())


    print('Epoch [{}/{}],Loss:{:.4f},Validation Loss:{:.4f},Accuracy:{:.2f},Validation Accuracy:{:.2f}'.format(epoch+1, num_epochs, loss.item(), val_loss, acc ,val_acc))

Epoch [1/10],Loss:0.2653,Validation Loss:12.2157,Accuracy:0.95,Validation Accuracy:0.98
Epoch [2/10],Loss:0.2834,Validation Loss:8.5202,Accuracy:0.97,Validation Accuracy:0.98
Epoch [3/10],Loss:0.2194,Validation Loss:6.7151,Accuracy:0.96,Validation Accuracy:0.95
Epoch [4/10],Loss:0.1373,Validation Loss:5.2936,Accuracy:0.99,Validation Accuracy:0.98
Epoch [5/10],Loss:0.1597,Validation Loss:5.1467,Accuracy:0.99,Validation Accuracy:0.98
Epoch [6/10],Loss:0.2147,Validation Loss:5.5634,Accuracy:0.99,Validation Accuracy:0.98
Epoch [7/10],Loss:0.1073,Validation Loss:4.8308,Accuracy:0.99,Validation Accuracy:0.98
Epoch [8/10],Loss:0.0294,Validation Loss:3.9383,Accuracy:1.00,Validation Accuracy:0.98
Epoch [9/10],Loss:0.0314,Validation Loss:4.6499,Accuracy:1.00,Validation Accuracy:0.98
Epoch [10/10],Loss:0.2402,Validation Loss:4.4777,Accuracy:0.99,Validation Accuracy:0.98


In [27]:
# скачаем набор картинок для проверки
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
